In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
temp = pd.read_csv('temp.csv')
temp.head()

,Year,Month,Day,Date,PPT.,Av temp,Tmax,Tmin
0,1901,1,1,01/01/1901,0.0,2.1,3.5,0.6
1,1901,1,2,02/01/1901,0.0,4.0,7.2,0.7
2,1901,1,3,03/01/1901,0.0,3.0,6.7,-0.7
3,1901,1,4,04/01/1901,0.0,3.7,7.9,-0.6
4,1901,1,5,05/01/1901,0.0,-0.3,1.2,-1.7
